In [14]:
import numpy as np
import pandas as pd

# 读取数据
MV = pd.read_csv('季_相对价值指标.csv')
MV = MV.iloc[2:]
MV = MV.rename(columns={'市值A': 'MV'})

def My_YM(Set, Col):
    Set['DATE'] = Set[Col].astype(str).replace('-', '', regex=True)
    Set['Yearmon'] = Set['DATE'].astype(int) // 100

My_YM(MV, '会计期间')

# 将 MV 转换为浮点数，同时处理非数字值
MV['MV'] = pd.to_numeric(MV['MV'], errors='coerce')

# 过滤掉无效值（小于等于0的值）
MV = MV[MV['MV'] > 0]

# 计算对数
MV['mve'] = np.log(MV['MV']).shift(1)

# 分组并计算均值
MV2 = MV[['mve', '行业代码', '会计期间']].groupby(['行业代码', '会计期间']).mean().reset_index()
MV2 = MV2.rename(columns={'mve': 'mveI'})

# 再次合并并计算差异
MV3 = pd.merge(MV, MV2, on=['行业代码', '会计期间'], how='left')
MV3['mve_ia'] = MV3['mve'] - MV3['mveI']

# 过滤掉 NaN 值
MV3 = MV3[['证券代码', '会计期间', 'mve', 'mve_ia']]
MV3 = MV3.dropna()
MV3

,证券代码,会计期间,mve,mve_ia
1,1,2001-06-30,25.154830,-0.117096
2,1,2001-12-31,25.357584,-0.201941
3,1,2002-03-31,25.626792,-0.117263
4,1,2002-06-30,25.539287,0.358479
5,1,2002-09-30,25.753536,-0.336585
...,...,...,...,...
244672,900957,2022-03-31,21.119533,-2.967366
244673,900957,2022-06-30,21.073693,-3.122162
244674,900957,2022-09-30,21.125145,-3.028358
244675,900957,2022-12-31,21.145669,-2.970402


In [15]:
import numpy as np
import pandas as pd

MV = pd.read_csv('季_相对价值指标.csv')
MV = MV.iloc[2:]
MV = MV.rename(columns={'市值A': 'MV'})

def My_YM(Set, Col):
    Set['DATE'] = Set[Col].astype(str).replace('-', '', regex=True)
    Set['Yearmon'] = Set['DATE'].astype(int) // 100

My_YM(MV, '会计期间')

# 将 MV 转换为浮点数，同时处理非数字值，保留 NaN
MV['MV'] = pd.to_numeric(MV['MV'], errors='coerce')

# 计算对数
MV['mve'] = np.log(MV['MV']).shift(1)  # NaN 会自动填充在计算中

# 分组并计算均值
MV2 = MV[['mve', '行业代码', '会计期间']].groupby(['行业代码', '会计期间']).mean().reset_index()
MV2 = MV2.rename(columns={'mve': 'mveI'})

# 再次合并并计算差异
MV3 = pd.merge(MV, MV2, on=['行业代码', '会计期间'], how='left')
MV3['mve_ia'] = MV3['mve'] - MV3['mveI']  # 计算差异

# 保留所有列，将 NaN 作为缺失值
MV3 = MV3[['证券代码', '会计期间', 'mve', 'mve_ia']]

# 不调用 dropna()，保留 NaN
# MV3 = MV3.dropna()  

MV3

/root/miniconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,证券代码,会计期间,mve,mve_ia
0,1,2000-09-30,NaN,NaN
1,1,2000-12-31,NaN,NaN
2,1,2001-03-31,25.154830,NaN
3,1,2001-06-30,NaN,NaN
4,1,2001-09-30,25.357584,NaN
...,...,...,...,...
571124,920118,2023-09-30,NaN,NaN
571125,920118,2023-12-31,NaN,NaN
571126,920118,2024-03-31,NaN,NaN
571127,920118,2024-06-30,NaN,NaN


In [16]:
MV3.to_csv('季_46_47.csv', index=False, encoding='utf-8-sig')